# Toronto Neighborhood Clustering - IBM Data Science Capstone
#### by fil.coutinho

## 1. Dataframe Preparation

In [ ]:
# The code was removed by Watson Studio for sharing.

In [5]:
body = client_61bf4a96af25446cb70fbd99af7c40a1.get_object(Bucket='ibmdatasciencecapstone-donotdelete-pr-butz1ybluhym17',Key='toronto_neigh.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_toronto = pd.read_excel(body)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [18]:
df_toronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

#Drop 'Not Assigned' rows
df_toronto.drop(df_toronto.loc[df_toronto['Borough']=='Not assigned'].index, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [27]:
count_postal = df_toronto.groupby(['PostalCode']).count()
count_postal.head()

,Borough,Neighborhood
PostalCode,,
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1


In [28]:
#Check for duplicate postal codes
count_postal.loc[count_postal['Neighborhood'] > 1]

,Borough,Neighborhood
PostalCode,,
